### Cadê Covid - Modelagem Modelo Preditivo

In [6]:
import pandas as pd

In [7]:
state_density_df = pd.read_csv("../data_source/states_density.csv")

In [8]:
caso_full = pd.read_csv('../data_source/brasil_io_caso_full.csv')

In [11]:
caso_full.head()

,city,city_ibge_code,date,epidemiological_week,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,new_confirmed,new_deaths,order_for_place,place_type,state
0,Acrelândia,1200013.0,2020-07-15,29,15256.0,False,True,227,1487.93917,2020-07-14,0.0132,3,0,0,109,city,AC
1,Assis Brasil,1200054.0,2020-07-15,29,7417.0,False,True,227,3060.53661,2020-07-14,0.0220,5,0,0,76,city,AC
2,Brasiléia,1200104.0,2020-07-15,29,26278.0,False,True,534,2032.11812,2020-07-14,0.0150,8,0,0,68,city,AC
3,Bujari,1200138.0,2020-07-15,29,10266.0,False,True,270,2630.04091,2020-07-14,0.0148,4,0,0,99,city,AC
4,Capixaba,1200179.0,2020-07-15,29,11733.0,False,True,181,1542.65746,2020-07-14,0.0387,7,0,0,69,city,AC


In [12]:
caso_full.dtypes

city                                              object
city_ibge_code                                   float64
date                                              object
epidemiological_week                               int64
estimated_population_2019                        float64
is_last                                             bool
is_repeated                                         bool
last_available_confirmed                           int64
last_available_confirmed_per_100k_inhabitants    float64
last_available_date                               object
last_available_death_rate                        float64
last_available_deaths                              int64
new_confirmed                                      int64
new_deaths                                         int64
order_for_place                                    int64
place_type                                        object
state                                             object
dtype: object